## Crear una base de datos nueva

Se extraerà concepto de la fuente original: https://docs.python.org/2/library/sqlite3.html para comprender en totalidad lo aprendido en Py4e.com 

In [52]:
import sqlite3
conn = sqlite3.connect('Ages')#Se conecta la base de datos con la direccion entre parentesis
#Seleccionar el cursor para realizar la consulta
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS Ages')
cur.execute('''CREATE TABLE Ages
             (name VARCHAR(128), age INTEGER)''')

cur.execute("INSERT INTO Ages (name, age) VALUES (?, ?)",  ('Dylan', 22))
cur.execute("INSERT INTO Ages (name, age) VALUES (?, ?)",  ('Omar', 37))
cur.execute("INSERT INTO Ages (name, age) VALUES (?, ?)",  ('Riley', 18))
cur.execute("INSERT INTO Ages (name, age) VALUES (?, ?)",  ('Mili', 22))

conn.commit()
conn.close()

#Source:https://docs.python.org/2/library/sqlite3.html

In [ ]:
import sqlite3
conn = sqlite3.connect('Counting_Organization')#Se conecta la base de datos con la direccion entre parentesis
#Seleccionar el cursor para realizar la consulta
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS count')
cur.execute('''CREATE TABLE count
             (email VARCHAR(128), count INTEGER)''')

cur.execute("INSERT INTO Ages (email, count) VALUES (?, ?)",  ('k', v))
conn.commit()
conn.close()

## Extrayendo datos de una base de datos

In [ ]:
import sqlite3

conn = sqlite3.connect('Ages') 
cur = conn.cursor()

print('Ages:')
cur.execute('SELECT name, age FROM Ages')
cur.execute('SELECT name ,age FROM Ages ORDER BY age') #Ordenar age de menor a mayor. 
#cur.execute('DELETE FROM Ages WHERE age = 18')  #Eliminar una fila
for row in cur:
     print(row)

#cur.execute('DELETE FROM name WHERE age < 100')

cur.close()
print(type(row))
print(type(cur))

## Counting Organizations

Bienvenido Ariel Chitay de Python para todos

Para obtener crédito por esta tarea, realice las siguientes instrucciones y cargue su base de datos SQLite3 aquí:
(Debe tener un sufijo .sqlite)

Sugerencia: el recuento máximo de la organización es 536.
No necesita exportar o convertir la base de datos, simplemente cargue el archivo .sqlite que crea su programa. Vea el código de ejemplo para el uso de la declaración connect ().

Organizaciones de conteo
Esta aplicación leerá los datos del buzón (mbox.txt) y contará el número de mensajes de correo electrónico por organización (es decir, el nombre de dominio de la dirección de correo electrónico) utilizando una base de datos con el siguiente esquema para mantener los conteos.

CREATE TABLE Counts (org TEXT, count INTEGER)
Cuando haya ejecutado el programa en mbox.txt, cargue el archivo de base de datos resultante anterior para calificar.
Si ejecuta el programa varias veces en pruebas o con diferentes archivos, asegúrese de vaciar los datos antes de cada ejecución.

Puede utilizar este código como punto de partida para su aplicación: http://www.py4e.com/code3/emaildb.py.

El archivo de datos para esta aplicación es el mismo que en asignaciones anteriores: http://www.py4e.com/code3/mbox.txt.

Debido a que el código de muestra está utilizando una instrucción UPDATE y está confirmando los resultados en la base de datos, ya que cada registro se lee en el ciclo, puede llevar unos minutos procesar todos los datos. El compromiso insiste en escribir completamente todos los datos en el disco cada vez que se llama.

El programa se puede acelerar mucho moviendo la operación de confirmación fuera del ciclo. En cualquier programa de base de datos, hay un equilibrio entre la cantidad de operaciones que ejecuta entre confirmaciones y la importancia de no perder los resultados de las operaciones que aún no se han confirmado.

In [2]:
import string
import os
import sqlite3
conn = sqlite3.connect('counting.sqlite')#Se conecta la base de datos con la direccion entre parentesis
#Seleccionar el cursor para realizar la consulta
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS count')
cur.execute('''CREATE TABLE count
             (email VARCHAR(128), count INTEGER)''')

try:
    fhand = open('mbox.txt')	
except:
    print('File cannot be opened:', fname)
    exit()

counts = dict()
for line in fhand:
    if line.startswith('From:'):
        word = line.split()[1]
        if word not in counts:
            counts[word] = 1
        else:
            counts[word] += 1


for k,v in counts.items():
    cur.execute("INSERT INTO count (email, count) VALUES (?, ?)",  (str(k), v))    
    

cur.execute('SELECT email ,count FROM count ORDER BY count') #Ordenar age de menor a mayor.
for row in cur:
     print(row)
        
conn.commit()
conn.close()

NameError: name 'fname' is not defined

In [ ]:
import sqlite3

conn = sqlite3.connect('emaildb.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Counts')

cur.execute('''
CREATE TABLE Counts (email TEXT, count INTEGER)''')

fname = input('Enter file name: ')
if (len(fname) < 1): fname = 'mbox-short.txt'
fh = open(fname)
for line in fh:
    if not line.startswith('From: '): continue
    pieces = line.split()
    email = pieces[1]
    cur.execute('SELECT count FROM Counts WHERE email = ? ', (email,))
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (email, count)
                VALUES (?, 1)''', (email,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE email = ?',
                    (email,))
    conn.commit()

# https://www.sqlite.org/lang_select.html
sqlstr = 'SELECT email, count FROM Counts ORDER BY count DESC LIMIT 10'

for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])

cur.close()

In [3]:
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Genre;
DROP TABLE IF EXISTS Track;
CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);
CREATE TABLE Genre (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name TEXT UNIQUE
);
CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);
CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id INTERGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')



fname = 'Library.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')
print('Dict count:', len(all))
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    genre = lookup(entry, 'Genre')
    artist = lookup(entry, 'Artist')
    album = lookup(entry, 'Album')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')

    if name is None or artist is None or album is None or genre is None: 
        continue

    print(name, genre, artist, album, count, rating, length)

    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ? )''', 
        ( name, album_id, length, rating, count ) )

    cur.execute('''INSERT OR IGNORE INTO Genre (name) 
        VALUES ( ?)''', ( genre, ) )
    cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
    genre_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Track
        (title, genre_id, album_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ?, ? )''', 
        ( name, genre_id, album_id, length, rating, count ) )

    conn.commit()

sqlstr = 'SELECT Track.title, Artist.name, Album.title, Genre.name FROM Track JOIN Genre JOIN Album JOIN Artist ON Track.genre_id = Genre.id AND Track.album_id = Album.id AND Album.artist_id = Artist.id ORDER BY Artist.name LIMIT 3'

spreadsheet = cur.execute(sqlstr)

for row in spreadsheet:
    print(row[0], row[1], row[2], row[3])

cur.close()

Dict count: 404
Another One Bites The Dust Rock Queen Greatest Hits 55 100 217103
Asche Zu Asche Industrial Rammstein Herzeleid 79 100 231810
Beauty School Dropout Soundtrack Various Grease 48 100 239960
Black Dog Rock Led Zeppelin IV 109 100 296620
Bring The Boys Back Home Rock Pink Floyd The Wall [Disc 2] 33 100 87118
Circles Funk Bryan Lee Blues Is 54 60 355369
Comfortably Numb Rock Pink Floyd The Wall [Disc 2] 36 100 384130
Crazy Little Thing Called Love Rock Queen Greatest Hits 38 100 163631
Electric Funeral Metal Black Sabbath Paranoid 44 100 293015
Fat Bottomed Girls Rock Queen Greatest Hits 38 100 257515
For Those About To Rock (We Salute You) Rock AC/DC Who Made Who 84 100 353750
Four Sticks Rock Led Zeppelin IV 84 100 284421
Furious Angels Soundtrack Rob Dougan The Matrix Reloaded 54 100 330004
Gelle Blues/R&B Bryan Lee Blues Is 45 60 199836
Going To California Rock Led Zeppelin IV 100 100 215666
Grease Soundtrack Various Grease 42 100 205792
Hand of Doom Metal Black Sabbath 

The Immigrant New Age Joanie Madden Natural Wonders Music Sampler 1999 350 None 362631
Pahrump-Big Water New Age Cusco Natural Wonders Music Sampler 1999 342 None 191634
Seeker's Quest New Age Cheryl Gunn Natural Wonders Music Sampler 1999 337 None 236434
Floating To Forever New Age Dean Everson Natural Wonders Music Sampler 1999 337 None 351686
Open Road Jazz Jeff Bailey Relaxing Jazz 10 None 318563
Ruby Alternative & Punk Kaiser Chiefs Yours Truly, Angry Mob 13 None 205008
The Angry Mob Alternative & Punk Kaiser Chiefs Yours Truly, Angry Mob 41 None 288313
Heat Dies Down Alternative & Punk Kaiser Chiefs Yours Truly, Angry Mob 12 None 237061
Highroyds Alternative & Punk Kaiser Chiefs Yours Truly, Angry Mob 10 None 199575
Love's Not A Competition (But I'm Winning) Alternative & Punk Kaiser Chiefs Yours Truly, Angry Mob 11 None 197799
Thank You Very Much Alternative & Punk Kaiser Chiefs Yours Truly, Angry Mob 11 None 157753
I Can Do It Without You Alternative & Punk Kaiser Chiefs Yours 

Ian Horrocks: Standardizing OWL Podcast IEEE Computer Society Computing Conversations 1 None 572212
Katie Hafner: The Origins of the Internet Podcast IEEE Computer Society Computing Conversations 1 None 652460
Larry Smarr: Building Mosaic Podcast IEEE Computer Society Computing Conversations None None 894693
Len Kleinrock: The Theory of Packets Podcast IEEE Computer Society Computing Conversations 1 None 674742
Mitchell Baker: The Mozilla Foundation Podcast IEEE Computer Society Computing Conversations None None 994246
Pooja Sankar: Building the Piazza Collaboration System Podcast IEEE Computer Society Computing Conversations None None 496404
Van Jacobson: Content-Centric Networking Podcast IEEE Computer Society Computing Conversations None None 780251
The Apache Software Foundation Podcast IEEE Computer Society Computing Conversations None None 542484
A Brief History of Packets Podcast IEEE Computer Society Computing Conversations None None 1004643
Discovering JavaScript Object Notati

Song for Someone Rock U2 Songs of Innocence None None 226763
The Miracle (Of Joey Ramone) Rock U2 Songs of Innocence None None 255382
Iris (Hold Me Close) Rock U2 Songs of Innocence None None 319457
Goodbye Daughters of the Revolution Rock The Black Crowes Warpaint None None 304147
Walk Believer Walk Rock The Black Crowes Warpaint None None 280253
Choice Hotels International Reality TV Undercover Boss Undercover Boss, Season 2 1 None 2614047
Microchip Dance & Electronic Jason Farnham YouTube Audio Library None None 114336
Anant Agarwal Podcast IEEE Computer Society Computing Conversations None None 494000
For Those About To Rock (We Salute You) AC/DC Who Made Who Rock
Hells Bells AC/DC Who Made Who Rock
Shake Your Foundations AC/DC Who Made Who Rock


SyntaxError: invalid syntax (<ipython-input-2-7c4dfd4aa1ea>, line 57)